# gnomAD Reference PC Projection

## Download gnomAD Reference Data

https://gnomad.broadinstitute.org/data#v4-genetic-ancestry-group-classification

https://console.cloud.google.com/storage/browser/gcp-public-data--gnomad/release/4.0/pca?pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))

In [ ]:
!wget https://storage.googleapis.com/gcp-public-data--gnomad/release/4.0/pca/gnomad.v4.0.pca_loadings.ht /home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD

## Load gnomAD loadings

In [ ]:
import hail as hl
hl.init(
    tmp_dir='/tmp/hail',  # Use system tmp
    local='local[4]',
    quiet=False  # See full error messages
)


In [ ]:
loadings_ht = hl.read_table('/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/gnomad.v4.0.pca_loadings.ht')
loadings_df = loadings_ht.to_pandas()  # Convert to pandas

In [ ]:
loadings_df.to_csv('/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/gnomad_loadings.csv', index=False)

In [ ]:
loadings_df.head()

## Extract overlapping variants from Tapestry

In [ ]:
# Create variant list from gnomAD loadings
# Save as chr:pos:ref:alt format (or chr:pos format, check gnomAD structure)
import pandas as pd
loadings_df = pd.read_csv('/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/gnomad_loadings.csv')  # from Step 1
print(loadings_df.shape)


In [ ]:
import ast
def format_variant_id(row):
    """Convert locus and alleles to chr:pos:ref:alt format"""
    locus = str(row['locus']).replace("chr", "")  # Remove 'chr' prefix if present
    alleles_list = ast.literal_eval(row['alleles'])
    ref = alleles_list[0]
    alt = alleles_list[1]
    return f"{locus}:{ref}:{alt}"

loadings_df['variant_id'] = loadings_df.apply(format_variant_id, axis=1)
loadings_df.head()

In [ ]:
# parse the loadings array
loadings_df['loadings_parsed'] = loadings_df['loadings'].apply(ast.literal_eval)

# Extract PC columns
for i in range(16):  # gnomAD v4 has 16 PCs
    loadings_df[f'PC{i+1}'] = loadings_df['loadings_parsed'].apply(lambda x: x[i])

# Verify
print(loadings_df[['variant_id', 'PC1', 'PC2', 'PC3']].head())

In [ ]:
variants_path = '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/gnomad_variants.txt'
loadings_df[['variant_id']].to_csv(variants_path, index=False, header=False)

In [ ]:
import subprocess

# Extract these variants from your Tapestry data
# Run plink2 as a subprocess
subprocess.run([
  "plink2",
  "--pfile", "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_wide/genome_qc",
  "--extract", variants_path,
  "--export", "A",
  "--out", "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_overlap/tapestry_gnomad_overlap"
], check=True)

# --export A creates .raw file (additive coding: 0/1/2 genotypes)

## Load Tapestry genotypes and align with loadings

In [ ]:
# Create score file
score_df = loadings_df[['variant_id'] + [f'PC{i}' for i in range(1, 17)]].copy()

# Add allele column (which allele the score applies to)
score_df['allele'] = loadings_df['alleles'].apply(lambda x: ast.literal_eval(x)[1])  # alt allele

# Reorder: variant_id, allele, PC1, PC2, ...
score_df = score_df[['variant_id', 'allele'] + [f'PC{i}' for i in range(1, 17)]]

# Save (tab-separated, no header for plink)
score_path = '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/gnomad_loadings.score'
score_df.to_csv(score_path, sep='\t', index=False, header=False)

In [ ]:
score_df.head()

In [ ]:
subprocess.run([
  "plink2",
  "--pfile", "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_wide/genome_qc",
  "--extract", variants_path,
  "--score", score_path, "1", "2", "header",
  "--score-col-nums", "3-18",
  "--out", "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_overlap/tapestry_projected_pcs"
], check=True)

In [ ]:
read_path = "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_overlap/tapestry_projected_pcs.sscore"
pcs = pd.read_csv(read_path, sep='\s+')

# Keep only the AVG columns (these are the projected PCs)
pc_cols = [col for col in pcs.columns if '_AVG' in col]
pcs_clean = pcs[['#IID'] + pc_cols].copy()

# Rename columns
pcs_clean.columns = ['IID'] + [f'PC{i}' for i in range(1, 17)]

# Save
pcs_clean.to_csv('tapestry_projected_pcs.csv', index=False)

print(f"Projected {len(pcs_clean)} samples onto 16 PCs")
pcs_clean.head()

## Validation Plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')

# Create figure with multiple subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# PC1 vs PC2 (most informative)
ax1 = axes[0, 0]
ax1.scatter(pcs_clean['PC1'], pcs_clean['PC2'], alpha=0.3, s=5)
ax1.set_xlabel('PC1', fontsize=12)
ax1.set_ylabel('PC2', fontsize=12)
ax1.set_title('PC1 vs PC2', fontsize=14)
ax1.grid(alpha=0.3)

# PC1 vs PC3
ax2 = axes[0, 1]
ax2.scatter(pcs_clean['PC1'], pcs_clean['PC3'], alpha=0.3, s=5)
ax2.set_xlabel('PC1', fontsize=12)
ax2.set_ylabel('PC3', fontsize=12)
ax2.set_title('PC1 vs PC3', fontsize=14)
ax2.grid(alpha=0.3)

# PC2 vs PC3
ax3 = axes[1, 0]
ax3.scatter(pcs_clean['PC2'], pcs_clean['PC3'], alpha=0.3, s=5)
ax3.set_xlabel('PC2', fontsize=12)
ax3.set_ylabel('PC3', fontsize=12)
ax3.set_title('PC2 vs PC3', fontsize=14)
ax3.grid(alpha=0.3)

# PC1 vs PC4
ax4 = axes[1, 1]
ax4.scatter(pcs_clean['PC1'], pcs_clean['PC4'], alpha=0.3, s=5)
ax4.set_xlabel('PC1', fontsize=12)
ax4.set_ylabel('PC4', fontsize=12)
ax4.set_title('PC1 vs PC4', fontsize=14)
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('gnomad_projected_pcs_overview.png', dpi=300)
plt.show()